In [1]:
import oracledb
import pyodbc


SQL_CREDENTIALS = {
    "driver": "ODBC Driver 17 for SQL Server",
    "server": "172.17.17.22,54312",
    "database": "RISKDB",
    "username": "risk_technology_dev",
    "password": "tTcnjl6T",
}


ORACLE_CREDENTIALS = {
        "user": "NishanovII",
        "password": "hfSAjVOqH0Q5",
        "dsn": oracledb.makedsn("192.168.81.115", "1521", service_name="orcl1")
}

def get_mssql_connection():
    creds = SQL_CREDENTIALS
    return pyodbc.connect(
        f"Driver={{{creds['driver']}}};"
        f"Server={creds['server']};"
        f"Database={creds['database']};"
        f"UID={creds['username']};"
        f"PWD={creds['password']};"
    )

def get_oracle_connection():
    creds = ORACLE_CREDENTIALS
    oracledb.init_oracle_client()
    return oracledb.connect(user=creds["user"], password=creds["password"], dsn=creds["dsn"])


In [6]:
connection = get_mssql_connection()
cursor = connection.cursor()


cursor.execute("select @@VERSION")
version = cursor.fetchall()
connection.commit()
cursor.close()
print(version)

[('Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tEnterprise Edition: Core-based Licensing (64-bit) on Windows Server 2022 Standard 10.0 <X64> (Build 20348: ) (Hypervisor)\n',)]


In [9]:
connection = get_oracle_connection()
cursor = connection.cursor()


cursor.execute("""SELECT version FROM PRODUCT_COMPONENT_VERSION
WHERE product LIKE 'Oracle Database%'""")
version = cursor.fetchall()
connection.commit()
cursor.close()
print(version)

[('11.2.0.4.0',)]


In [4]:
from datetime import datetime, timedelta

In [5]:
def get_date_range():

    today = datetime.today()
    first_day_of_current_month = today.replace(day=1)
    end_date = first_day_of_current_month.strftime('%d-%m-%Y')
    start_date = (first_day_of_current_month - timedelta(days=90)).replace(day=1).strftime('%d-%m-%Y')
    return start_date, end_date

In [6]:
get_date_range()

('01-10-2024', '01-01-2025')

In [3]:
products = '24, 34, 32'
print(f"AND t.K_VID_CRED in ({products})")

AND t.K_VID_CRED in (24, 34, 32)
